In [1]:
import torch

# デフォルトボックスからバウンディングボックスへの変換

In [2]:
def decode(loc, dbox_list):
    '''
    locネットワークが出力するオフセット情報を使用してDBoxをBBoxに変換する

    Parameters:
      loc(Tensor):
        locが出力する(8732,4)の形状のテンソル
        8,732個のDBoxのオフセット情報(Δcx, Δcy, Δwidth, Δheight)
      dbox_list(Tensor):
        DBoxの情報(cx, cy, width, height)を格納した(8732,4)のテンソル
        
    Returns(Tensor):
      BBoxの情報(xmin, ymin, xmax, ymax)を格納したテンソル(8732, 4)
    '''
    # DBoxにlocのオフセットを適用してBBoxの(cx, cy, width, height)を求める
    # 変数boxesの形状は(8732, 4)
    boxes = torch.cat((
        # cx = cx_d + 0.1Δcx ･ w_d
        # cy = cy_d + 0.1Δcy ･ h_d
        dbox_list[:, :2] + loc[:, :2] * 0.1 * dbox_list[:, 2:],
        # w = w_d ･ exp(0.2Δw)
        # h = h_d ･ exp(0.2Δh)
        dbox_list[:, 2:] * torch.exp(loc[:, 2:] * 0.2)
        ),dim=1)

    # BBoxの情報(cx, cy, width, height)を(xmin, ymin, xmax, ymax)に変換
    boxes[:, :2] -= boxes[:, 2:] / 2  # (cx, cy)を(xmin,ymin)にする
    boxes[:, 2:] += boxes[:, :2]      # (width, height)を(xmax,ymax)にする

    return boxes

デコード関数はモデルが学習したデフォルトボックスの位置情報をバウンディングボックスに変換する関数  
ssdのネットワークにはデフォルトボックスの位置情報を持っている(cx_d, cy_d, w_d, h_d)  
これに対して、物体中心へのデフォルトボックスのオフセット値を出力する(⊿cx, ⊿cy, ⊿w, ⊿y)  

この2つの情報を元にバウンディングボックスの座標を計算する  
$$
    cx = cx\_d(1+0.1\Delta cx) \\
    cy = cy\_d(1+0.1\Delta cy) \\
    w = w\_d \times exp(0.2\Delta w) \\
    h = h\_d \times exp(0.2\Delta h)
$$

これは理論的に決まっているのではなく、このように元のバウンディングボックスの座標を変換するように学習することを定めている(なぜこうなのかは不明)

# Non-Maximum Supression
バウンディングの被りの処理  
<img src="https://cvml-expertguide.net/wp-content/uploads/2022/08/07e652a1b89611f1c0aa98bc50c80ac6.png" width=800 />

In [3]:
def nonmaximum_suppress(
        boxes, scores, overlap=0.5, top_k=200): ###### nm_suppressionを変更、オリジナルの0.5に戻す######
    '''1つの物体に対して1つのBBoxだけを残す
    
    画像分類のクラスごとにNon-Maximum Suppressionを実施
    クラス単位で抽出された確信度0.01以上のboxesから同一の物体に対する被り度
    （IoU値）が大きいBBoxを集めて、その中で最大の確信度を持つBBoxだけを取り出す

    Parameters:
      boxes(Tensor):
        1クラスあたり8,732個のBBoxのうち、確信度0.01を超えたDBoxの座標情報
        テンソルの形状は(1クラスにつき確信度0.01を超えたDBoxの数, 4)
      scores(Tensor):
          confネットワークの出力(DBoxの各クラスの確信度)からクラスごとに
          確信度の閾値0.01を超えるBBoxの確信度だけを抜き出したもの
          テンソルの形状は(1クラスにつき確信度0.01を超えたBBoxの数, )
      overlap(float):
        被り度合い（IoU値）の基準にする値
        overlapが0.5以上である場合に、それらのバウンディングボックスは
        同じ物体に対するバウンディングボックスと判断する
      top_k(int)
　      scoresから確信度が高い順にサンプルを取り出す際の、取り出すサンプルの数

    Returns:
      keep(Tensor): 画像中に存在するBBoxのインデックスが格納される
      count(int):  画像中に存在するBBoxの数が格納される
    '''
    # NMSを通過したBBoxの数を保持する変数の初期化
    count = 0
    # scoresと同じ形状の0で初期化したテンソルを生成
    # keepの形状は(1クラスにつき確信度0.01を超えたBBoxの数,)
    keep = scores.new(scores.size(0)).zero_().long()

    # 各BBoxの面積areaを計算
    # areaの形状は(確信度0.01を超えるBBoxの数,)
    x1 = boxes[:, 0] # x軸最小値
    y1 = boxes[:, 1] # y軸最小値
    x2 = boxes[:, 2] # x軸最大値
    y2 = boxes[:, 3] # y軸最大値
    area = torch.mul(x2 - x1, y2 - y1) # torch.mulで底辺×高さを求める

    # boxesのコピーをBBox情報の要素の数だけ作成
    # BBoxの被り度(IoU)の計算の際に使用する
    tmp_x1 = boxes.new()
    tmp_y1 = boxes.new()
    tmp_x2 = boxes.new()
    tmp_y2 = boxes.new()
    tmp_w = boxes.new()
    tmp_h = boxes.new()

    # socreを昇順(確信度が低い方から)に並び変える
    v, idx = scores.sort(0) # idxに元の要素のインデックスのリストを格納

    # idxの上位top_k個（200個）のBBoxのインデックスを取り出す
    # 200個存在しない場合もある
    idx = idx[-top_k:]

    # idx(初期の要素数top_k個（200個）)の要素数が0でない限りループ
    while idx.numel() > 0:
        i = idx[-1]  # 最大の確信度(conf値)のインデックスを取得
        
        # keepの形状は(1クラスにつき確信度0.01を超えたBBoxの数,)
        # keepのインデックスcountの位置に最大確信度(conf値)のインデックス値を格納
        # このインデックスのBBoxと被りが大きいBBoxを以下の処理で取り除く
        keep[count] = i
        # keepのインデックスを1増やす
        count += 1

        # idxの要素数を取得し、1(最後のBBox)であればループを抜ける
        if idx.size(0) == 1:
            break

        ### Non-Maximum Suppressionの処理を開始 ###
        # 昇順に並んでいるscoresのインデックスの末尾を除外する
        idx = idx[:-1]

        # idxの昇順スコアのインデックス値を使ってBBoxの座標情報xmin, ymin, xmax, ymaxの
        # 情報を抽出してtmp_x1、tmp_y1、tmp_x2、tmp_y2に格納
        # index_select(入力Tensor,
        #              対象の次元,
        #              抽出する要素のインデックス,
        #              out=出力Tensor名)
        torch.index_select(x1, 0, idx, out=tmp_x1) # 昇順スコアに対応するxminの並び
        torch.index_select(y1, 0, idx, out=tmp_y1) # 昇順スコアに対応するyminの並び
        torch.index_select(x2, 0, idx, out=tmp_x2) # 昇順スコアに対応するxmaxの並び
        torch.index_select(y2, 0, idx, out=tmp_y2) # 昇順スコアに対応するymaxの並び

        # idxに残っているBBoxのxmin, ymin, xmax, ymaxの下限値を
        # それぞれインデックスi(確信度最上位のBBox）の値までに切り詰める
        # torch.clamp(入力Tensor,
        #             min=切り詰める下限値,
        #             max=切り詰める上限値,
        #             out=出力Tensor名)
        tmp_x1 = torch.clamp(tmp_x1, min=x1[i]) # xminの下限値を切り詰める
        tmp_y1 = torch.clamp(tmp_y1, min=y1[i]) # yminの下限値を切り詰める
        tmp_x2 = torch.clamp(tmp_x2, max=x2[i]) # xmaxの下限値を切り詰める
        tmp_y2 = torch.clamp(tmp_y2, max=y2[i]) # xmaxの下限値を切り詰める

        # tmp_wとtmp_hのテンソルの形状をそれぞれtmp_x2、tmp_y2と同じ形状にする
        tmp_w.resize_as_(tmp_x2)
        tmp_h.resize_as_(tmp_y2)

        # tmp_x1, tmp_y1, tmp_x2, tmp_y2を使って重なる部分の幅と高さを求め
        # tmp_wとtmp_hに代入する
        tmp_w = tmp_x2 - tmp_x1
        tmp_h = tmp_y2 - tmp_y1

        # 幅や高さが負の値になっていたら0にする
        tmp_w = torch.clamp(tmp_w, min=0.0)
        tmp_h = torch.clamp(tmp_h, min=0.0)

        # intersect(交差)部分の面積(A ∩ B)を求める
        inter = tmp_w*tmp_h

        # IoU = intersect部分 / (area(a) + area(b) - intersect部分)の計算
        # areaからidxに残っているすべてのBBoxの面積を取得
        rem_areas = torch.index_select(
            area, # 確信度0.01以上のすべてのBBoxの面積
            0,    # 処理対象の次元
            idx)  # 確信度上位200から現存するBBoxのインデックス値の並び
        # (BBoxの元の面積 - 交差部分の面積)+基準となるBBox(確信度最上位）の面積
        union = (rem_areas - inter) + area[i]  # A∪Bの面積
        IoU = inter/union # idxに残っているすべてのBBoxのIoUを求める

        # idxに残っているBBoxのうちIoUがoverlapより小さいものだけを残す
        # 同じ物体を囲むその他のBBoxがすべて取り除かれる
        idx = idx[IoU.le(overlap)]  # le()はoverlap以下の要素だけを残す

    # idxのBBoxが１個になりwhileループを抜けたら
    # 検出されたBBoxの数とBBoxを参照するためのインデックス値を返して終了
    return keep, count